In [4]:
# =============================
# SPEP gel quantification with LAZY NumPy
# - Tkinter/Pillow rectangle selection (no NumPy needed)
# - Lazy import NumPy only for linear fit (with pure-Python fallback)
# =============================

import os, math, statistics, json
from dataclasses import dataclass, asdict
from pathlib import Path
from typing import List, Tuple, Dict, Optional

# --- I/O (no NumPy) ---
from PIL import Image, ImageTk

# --- Tk UI for rectangle selection (no NumPy) ---
import tkinter as tk
from tkinter import filedialog, simpledialog, messagebox


In [5]:
# ------------- CONFIG -------------
IMAGE_PATH = ""  # leave empty to get a file picker; or set a path like "/path/gel.jpg"
BAND_NAMES_DEFAULT = ["Albumin", "Alpha1", "Alpha2", "Beta", "Gamma"]
SAVE_PREFIX = "SPEP_"
# ----------------------------------

# ------ Lazy NumPy loader (used only for linear regression) ------
def lazy_np():
    global np
    try:
        import numpy as np  # noqa
        return np
    except Exception:
        return None


In [6]:
# ---------- Data structures ----------
@dataclass
class Rect:
    x: float; y: float; w: float; h: float  # in original image coordinates

@dataclass
class Lane:
    lane_id: str
    lane_type: str  # "NEG", "POS", "SAMPLE"
    rect: Rect

@dataclass
class Band:
    lane_id: str
    band_name: str
    rect: Rect

In [7]:
# ---------- Image utils (Pillow) ----------
def load_image_gray(path: str) -> Image.Image:
    img = Image.open(path)
    if img.mode != "L":
        img = img.convert("L")
    return img

def crop(img: Image.Image, r: Rect) -> Image.Image:
    x0 = max(0, int(round(r.x)))
    y0 = max(0, int(round(r.y)))
    x1 = min(img.width, int(round(r.x + r.w)))
    y1 = min(img.height, int(round(r.y + r.h)))
    return img.crop((x0, y0, x1, y1))

def iod_intensity(roi: Image.Image) -> float:
    """
    Integrated optical density:
    - invert (band dark on bright background) -> signal = 255 - gray
    - background = median of 10% border pixels
    - return sum(max(pixel-bg, 0))
    """
    inv = Image.eval(roi, lambda p: 255 - p)
    w, h = inv.size
    px = list(inv.getdata())

    b = max(1, int(0.1 * min(w, h)))
    # top & bottom stripes
    top = px[: w * b]
    bot = px[-w * b:] if h > b else []
    # left & right stripes
    left = []
    right = []
    for yy in range(h):
        row0 = yy * w
        left.extend(px[row0 : row0 + b])
        right.extend(px[row0 + (w - b) : row0 + w])
    border = top + bot + left + right
    bg = statistics.median(border) if border else 0

    sig = 0.0
    for v in px:
        dv = v - bg
        if dv > 0:
            sig += dv
    return float(sig)


In [14]:
class RectSelector:
    """
    Tk canvas selector with keyboard shortcuts:
      - Drag: draw rectangle
      - Enter or Esc: Done
      - 'u' or Backspace: Undo last rectangle
    It also auto-scales so the button bar is visible on small screens.
    """
    def __init__(self, pil_image: Image.Image, title="Select rectangles"):
        self.img = pil_image
        self.root = tk.Tk()
        self.root.title(title)
        self.root.protocol("WM_DELETE_WINDOW", self.finish)

        # --- Fit within screen so buttons stay visible ---
        scr_w = self.root.winfo_screenwidth()
        scr_h = self.root.winfo_screenheight()
        max_w, max_h = int(scr_w * 0.9), int(scr_h * 0.85)  # leave room for buttons/titlebar

        scale = min(max_w / self.img.width, max_h / self.img.height, 1.0)
        self.scale = scale
        disp = self.img.resize((int(self.img.width * scale), int(self.img.height * scale)), Image.BILINEAR)
        self.tk_img = ImageTk.PhotoImage(disp)

        # Layout: canvas on top, buttons at bottom
        self.canvas = tk.Canvas(self.root, width=disp.width, height=disp.height, cursor="cross")
        self.canvas.grid(row=0, column=0, sticky="nsew")
        btn_frame = tk.Frame(self.root)
        btn_frame.grid(row=1, column=0, sticky="ew")

        # Make canvas expand but keep button row visible
        self.root.grid_rowconfigure(0, weight=1)
        self.root.grid_columnconfigure(0, weight=1)

        self.canvas.create_image(0, 0, image=self.tk_img, anchor="nw")
        # On-canvas hint
        self.canvas.create_text(10, 10, anchor="nw",
                                text="Drag to add rectangles • Enter/Esc = Done • u/Backspace = Undo",
                                fill="lime", font=("Helvetica", 12, "bold"))

        tk.Button(btn_frame, text="Undo last", command=lambda: self.undo()).pack(side=tk.LEFT, padx=6, pady=6)
        tk.Button(btn_frame, text="Done", command=self.finish).pack(side=tk.RIGHT, padx=6, pady=6)

        # Mouse bindings
        self.start = None
        self.rects = []          # canvas ids
        self.out_rects = []      # Rects in original-image coords
        self._current_rect = None

        self.canvas.bind("<ButtonPress-1>", self.on_press)
        self.canvas.bind("<B1-Motion>", self.on_drag)
        self.canvas.bind("<ButtonRelease-1>", self.on_release)

        # Keyboard shortcuts
        self.root.bind("<Return>", lambda e: self.finish())
        self.root.bind("<Escape>", lambda e: self.finish())
        self.root.bind("<BackSpace>", lambda e: self.undo())
        self.root.bind("u", lambda e: self.undo())

    def on_press(self, event):
        self.start = (event.x, event.y)
        self._current_rect = self.canvas.create_rectangle(event.x, event.y, event.x, event.y,
                                                          outline="lime", width=2)

    def on_drag(self, event):
        if self._current_rect is not None and self.start is not None:
            x0, y0 = self.start
            self.canvas.coords(self._current_rect, x0, y0, event.x, event.y)

    def on_release(self, event):
        if self._current_rect is None or self.start is None:
            return
        x0, y0 = self.start
        x1, y1 = event.x, event.y
        x, y = min(x0, x1), min(y0, y1)
        w, h = abs(x1 - x0), abs(y1 - y0)
        if w >= 2 and h >= 2:
            self.rects.append(self._current_rect)
            # map back to original coords
            rx, ry = x / self.scale, y / self.scale
            rw, rh = w / self.scale, h / self.scale
            self.out_rects.append(Rect(rx, ry, rw, rh))
        else:
            self.canvas.delete(self._current_rect)
        self._current_rect = None
        self.start = None

    def undo(self):
        if self.rects:
            rid = self.rects.pop()
            self.canvas.delete(rid)
            self.out_rects.pop()

    def finish(self):
        try:
            self.root.destroy()
        except tk.TclError:
            pass  # already closed

    def select(self) -> List[Rect]:
        self.root.mainloop()
        return self.out_rects


In [15]:
# ---------- Workflow helpers ----------
def select_lanes(img: Image.Image, label: str) -> List[Lane]:
    rs = RectSelector(img, title=f"Select {label} LANES (vertical rectangles), then click 'Done'")
    rects = rs.select()
    lanes = []
    for i, r in enumerate(rects, start=1):
        lanes.append(Lane(lane_id=f"{label}-{i}", lane_type=label, rect=r))
    return lanes

def select_bands_for_lane(full_img: Image.Image, lane: Lane, band_names: Optional[List[str]] = None) -> List[Band]:
    lane_img = crop(full_img, lane.rect)
    rs = RectSelector(lane_img, title=f"{lane.lane_id}: select band rectangles (top→bottom), then 'Done'")
    rects_local = rs.select()
    # translate to full image coords
    bands = []
    names = band_names if (band_names and len(band_names)==len(rects_local)) else [f"Band{i+1}" for i in range(len(rects_local))]
    for nm, rr in zip(names, rects_local):
        r_full = Rect(x=rr.x + lane.rect.x, y=rr.y + lane.rect.y, w=rr.w, h=rr.h)
        bands.append(Band(lane_id=lane.lane_id, band_name=nm, rect=r_full))
    return bands

def measure_all(img: Image.Image, bands: List[Band]) -> List[Dict]:
    rows = []
    for b in bands:
        roi = crop(img, b.rect)
        iod = iod_intensity(roi)
        rows.append(dict(lane_id=b.lane_id, band_name=b.band_name, iod_intensity=iod))
    return rows


In [16]:
# ---------- Calibration (lazy NumPy with pure-Python fallback) ----------
def linear_fit(x_list, y_list):
    """
    Return (a,b) minimizing ||a*x + b - y||^2.
    - Tries NumPy if available.
    - Falls back to closed-form pure Python otherwise.
    """
    np = lazy_np()
    if np is not None:
        x = np.asarray(x_list, dtype=float)
        y = np.asarray(y_list, dtype=float)
        A = np.vstack([x, np.ones_like(x)]).T
        a, b = np.linalg.lstsq(A, y, rcond=None)[0]
        return float(a), float(b)
    # Pure-Python least-squares for y = a x + b
    n = len(x_list)
    sx = sum(x_list); sy = sum(y_list)
    sxx = sum(v*v for v in x_list)
    sxy = sum(x_list[i]*y_list[i] for i in range(n))
    denom = (n * sxx - sx * sx)
    if abs(denom) < 1e-12:
        # fallback to proportional fit b=0
        a = sxy / (sxx + 1e-12)
        b = 0.0
        return a, b
    a = (n * sxy - sx * sy) / denom
    b = (sy - a * sx) / n
    return a, b

def calibrate_per_band(measured_rows: List[Dict], control_conc: List[Dict]):
    """Build models per band_name using provided control concentrations."""
    # index measurements
    from collections import defaultdict
    meas = defaultdict(dict)  # band -> lane_id -> intensity
    bands = set()
    for r in measured_rows:
        bands.add(r["band_name"])
        meas[r["band_name"]][r["lane_id"]] = r["iod_intensity"]

    # group control conc by band
    by_band = defaultdict(list)
    for c in control_conc:
        by_band[c["band_name"]].append(c)

    models = {}
    diagnostics = []
    for band in bands:
        ctrls = by_band.get(band, [])
        if not ctrls:
            continue
        x, y, lids = [], [], []
        for c in ctrls:
            lid = c["lane_id"]
            if lid in meas[band] and c["conc_g_dl"] is not None:
                x.append(meas[band][lid]); y.append(float(c["conc_g_dl"])); lids.append(lid)
        if len(x) >= 1:
            a, b = linear_fit(x, y)
            models[band] = (a, b)
            for xi, yi, lid in zip(x, y, lids):
                diagnostics.append(dict(band_name=band, lane_id=lid, iod_intensity=xi,
                                       conc_g_dl=yi, pred=a*xi + b, resid=yi - (a*xi + b)))
    return models, diagnostics

def predict_samples(measured_rows: List[Dict], models: Dict[str, Tuple[float,float]], sample_lane_ids: List[str]):
    out = []
    for r in measured_rows:
        if r["lane_id"] in sample_lane_ids:
            band = r["band_name"]
            a, b = models.get(band, (float('nan'), float('nan')))
            pred = a * r["iod_intensity"] + b if not (math.isnan(a) or math.isnan(b)) else float('nan')
            out.append(dict(lane_id=r["lane_id"], band_name=band,
                            iod_intensity=r["iod_intensity"], conc_g_dl_est=pred))
    return out

In [17]:
# ---------- Simple CSV writer (no pandas dependency) ----------
def write_csv(path: str, rows: List[Dict], header: Optional[List[str]] = None):
    import csv
    if not rows:  # write header only if provided
        if header:
            with open(path, "w", newline="") as f:
                w = csv.writer(f); w.writerow(header)
        return
    if header is None:
        header = list(rows[0].keys())
    with open(path, "w", newline="") as f:
        w = csv.DictWriter(f, fieldnames=header)
        w.writeheader()
        for r in rows:
            w.writerow(r)

In [18]:
# ===================== RUN =====================
# 1) Load image
if not IMAGE_PATH:
    root = tk.Tk(); root.withdraw()
    IMAGE_PATH = filedialog.askopenfilename(
        title="Choose gel image (.jpg/.png/.tif)",
        filetypes=[("Images", "*.jpg *.jpeg *.png *.tif *.tiff")]
    )
    root.update(); root.destroy()

if not IMAGE_PATH:
    raise SystemExit("No image selected.")

img = load_image_gray(IMAGE_PATH)
print("Loaded:", IMAGE_PATH, "size:", img.size)


Loaded: /Users/bonellpatinoescobar/Desktop/UCSF/Wiita Lab/Chemidoc images/ChemiDoc Images/ChemiDoc Images 2025-05-09_PTC3891/Sample 5 - Wiita Lab 2025-05-09 12h08m09s Coomassie Blue 0.432s(Coomassie Blue) - Sample 5.jpg size: (1830, 1464)


In [19]:
# 2) Select lanes
neg_lanes = select_lanes(img, "NEG")      # lanes 13-16
pos_lanes = select_lanes(img, "POS")      # lanes 17-20
sample_lanes = select_lanes(img, "SAMPLE")

all_lanes: List[Lane] = neg_lanes + pos_lanes + sample_lanes
if not all_lanes:
    raise SystemExit("No lanes selected.")


2025-11-06 21:10:24.966 python[36439:21065378] _TIPropertyValueIsValid called with 16 on nil context!
2025-11-06 21:10:24.967 python[36439:21065378] imkxpc_getApplicationProperty:reply: called with incorrect property value 16, bailing.
2025-11-06 21:10:24.967 python[36439:21065378] Text input context does not respond to _valueForTIProperty:


In [20]:
# 3) Select bands per lane
all_bands: List[Band] = []
for lane in all_lanes:
    # Try to use canonical SPEP band names if 5 bands are selected
    bands = select_bands_for_lane(img, lane, BAND_NAMES_DEFAULT)
    all_bands.extend(bands)

In [21]:
# 4) Measure intensities
measured = measure_all(img, all_bands)
print(f"Measured {len(measured)} band intensities.")

# Save ROIs and measurements
roi_lanes_rows = [dict(lane_id=l.lane_id, lane_type=l.lane_type, x=l.rect.x, y=l.rect.y, w=l.rect.w, h=l.rect.h) for l in all_lanes]
roi_bands_rows = [dict(lane_id=b.lane_id, band_name=b.band_name, x=b.rect.x, y=b.rect.y, w=b.rect.w, h=b.rect.h) for b in all_bands]
write_csv(SAVE_PREFIX + "lanes.csv", roi_lanes_rows)
write_csv(SAVE_PREFIX + "bands.csv", roi_bands_rows)
write_csv(SAVE_PREFIX + "measured_IOD.csv", measured)
print("Saved lanes, bands, and IOD to CSV.")


Measured 100 band intensities.
Saved lanes, bands, and IOD to CSV.


In [22]:
# 5) Enter control concentrations (g/dL)
# Generate a template to fill (if not already present)
tmpl = []
for l in neg_lanes + pos_lanes:
    lane_id = l.lane_id
    bnames = sorted({r["band_name"] for r in measured if r["lane_id"] == lane_id})
    for bn in bnames:
        tmpl.append(dict(lane_id=lane_id, band_name=bn, conc_g_dl=""))
tmpl_path = SAVE_PREFIX + "control_concentrations_template.csv"
write_csv(tmpl_path, tmpl)
print(f"A template CSV for control concentrations was written to: {tmpl_path}")


A template CSV for control concentrations was written to: SPEP_control_concentrations_template.csv


In [ ]:
# Ask whether to load a filled CSV or enter a few inline
root = tk.Tk(); root.withdraw()
resp = messagebox.askyesno("Controls", f"Did you already fill '{tmpl_path}'?\nYes = pick it now.\nNo = I will prompt you inline for one lane.")
root.update(); root.destroy()

controls = []
if resp:
    root = tk.Tk(); root.withdraw()
    csv_path = filedialog.askopenfilename(
        title="Select filled control concentrations CSV",
        filetypes=[("CSV", "*.csv")]
    )
    root.update(); root.destroy()
    if csv_path:
        import csv
        with open(csv_path, newline="") as f:
            for row in csv.DictReader(f):
                try:
                    val = float(row["conc_g_dl"])
                except Exception:
                    val = None
                controls.append(dict(lane_id=row["lane_id"], band_name=row["band_name"], conc_g_dl=val))
else:
    # Minimal inline entry for one NEG and/or POS lane (quick start)
    def prompt_lane_concs(lane: Lane):
        entries = []
        for bn in sorted({r["band_name"] for r in measured if r["lane_id"] == lane.lane_id}):
            root = tk.Tk(); root.withdraw()
            val = simpledialog.askstring("Control g/dL", f"{lane.lane_id} — {bn} (g/dL):")
            root.update(); root.destroy()
            if val is None or val.strip()=="":
                continue
            try:
                valf = float(val)
                entries.append(dict(lane_id=lane.lane_id, band_name=bn, conc_g_dl=valf))
            except Exception:
                pass
        return entries

    # pick the first available NEG lane, then first POS lane
    if neg_lanes:
        controls.extend(prompt_lane_concs(neg_lanes[0]))
    if pos_lanes:
        controls.extend(prompt_lane_concs(pos_lanes[0]))

if not controls:
    raise SystemExit("No control concentrations provided. Fill the template CSV and rerun from calibration.")

write_csv(SAVE_PREFIX + "controls_used.csv", controls)
print("Loaded control concentrations for", len(controls), "entries.")

In [1]:
# 6) Calibrate and predict
models, diag = calibrate_per_band(measured, controls)
write_csv(SAVE_PREFIX + "calibration_diagnostics.csv", diag)

sample_ids = [l.lane_id for l in sample_lanes]
pred = predict_samples(measured, models, sample_ids)
write_csv(SAVE_PREFIX + "sample_concentrations.csv", pred)

print("\n=== DONE ===")
print("Files written:")
print(" ", SAVE_PREFIX + "lanes.csv")
print(" ", SAVE_PREFIX + "bands.csv")
print(" ", SAVE_PREFIX + "measured_IOD.csv")
print(" ", SAVE_PREFIX + "control_concentrations_template.csv")
print(" ", SAVE_PREFIX + "controls_used.csv")
print(" ", SAVE_PREFIX + "calibration_diagnostics.csv")
print(" ", SAVE_PREFIX + "sample_concentrations.csv")

NameError: name 'calibrate_per_band' is not defined